### **Ajuste fino de Transformers con PyTorch y Hugging Face**


### **Configuración**


#### Instalando librerías necesarias


In [ ]:
# Todas las bibliotecas requeridas para este cuaderno están listadas a continuación.  
# !pip install -qy pandas==1.3.4 numpy==1.21.4 seaborn==0.9.0 matplotlib==3.5.0 torch=2.1.0+cu118  
# - Actualizar un paquete específico  
# !pip install pmdarima -U  
# - Actualizar un paquete a una versión concreta  
# !pip install --upgrade pmdarima==2.0.2  
# Nota: si tu entorno no admite `!pip install`, usa `!mamba install`


Necesitarás ejecutar la siguiente celdas para instalarlas:


In [ ]:
#!pip install transformers==4.42.1
#!pip install datasets # 2.20.0
#!pip install portalocker>=2.0.0
#!pip install torch==2.3.1
!pip install torchmetrics==1.4.0.post0
#!pip install numpy==1.26.4
#!pip install peft==0.11.1
#!pip install evaluate==0.4.2
#!pip install -q bitsandbytes==0.43.1
#!pip install accelerate==0.31.0
#!pip install torchvision==0.18.1


#!pip install trl==0.9.4
#!pip install protobuf==3.20.*
#!pip install matplotlib


#### Importación de librerías requeridas

_Se recomienda que importes todas las librerías necesarias en un solo lugar (aquí):_

* Nota: si obtienes un error al ejecutar la celda de abajo, intenta reiniciar el Kernel, ya que algunos paquetes necesitan un reinicio para que los cambios surtan efecto.

In [ ]:
import torch
from torchmetrics import Accuracy
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import AutoConfig,AutoModelForCausalLM,AutoModelForSequenceClassification,BertConfig,BertForMaskedLM,TrainingArguments, Trainer, TrainingArguments
from transformers import AutoTokenizer,BertTokenizerFast,TextDataset,DataCollatorForLanguageModeling
from transformers import pipeline
from datasets import load_dataset
from trl import SFTConfig,SFTTrainer, DataCollatorForCompletionOnlyLM


#import numpy as np
#import pandas as pd
from tqdm.auto import tqdm
import math
import time
import matplotlib.pyplot as plt
#import pandas as pd


# También puedes usar esta sección para suprimir las advertencias generadas por tu código:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

### **Ajuste fino(fine-tuning) supervisado con PyTorch**

El ajuste fino de Transformers, específicamente de BERT (Bidirectional Encoder Representations from Transformers), se refiere al proceso de entrenar un modelo BERT previamente entrenado en una tarea específica. BERT es un modelo de lenguaje de solo codificador que ha sido preentrenado en un gran corpus de texto para aprender representaciones contextuales de las palabras.

El ajuste fino de BERT implica tomar el modelo preentrenado y continuar entrenándolo con un conjunto de datos específico de la tarea, como análisis de sentimientos o respuesta a preguntas. Durante el ajuste fino, se actualizan y adaptan los parámetros del modelo BERT preentrenado a las particularidades de la tarea objetivo.

Este proceso es importante porque te permite aprovechar el conocimiento y la comprensión del lenguaje que BERT ha capturado y aplicarlo a diferentes tareas. Al ajustar BERT, puedes beneficiarte de su comprensión contextual del lenguaje y transferir ese conocimiento a problemas específicos de dominio o de tareas concretas. El ajuste fino permite que BERT aprenda a partir de un conjunto de datos etiquetado más pequeño y generalice bien a ejemplos no vistos, lo que lo convierte en una herramienta potente para diversas tareas de procesamiento de lenguaje natural. 

Ayuda a cerrar la brecha entre el preentrenamiento en un gran corpus y los requisitos específicos de las aplicaciones, mejorando en última instancia el rendimiento y la efectividad de los modelos en diversos escenarios del mundo real.


#### **Preparación del conjunto de datos**

El conjunto de datos de reseñas de Yelp es un recurso ampliamente utilizado en investigación de procesamiento de lenguaje natural (NLP) y análisis de sentimientos. Consta de reseñas de usuarios y metadatos asociados de la plataforma Yelp, que es un sitio popular para evaluar y calificar negocios locales como restaurantes, hoteles y tiendas.

El conjunto de datos incluye 6 990 280 reseñas escritas por usuarios de Yelp, que abarcan una amplia variedad de negocios y ubicaciones. Cada reseña generalmente contiene el texto de la opinión junto con la calificación en estrellas otorgada por el usuario (de 1 a 5).

El objetivo en este cuaderno es ajustar finamente un modelo BERT preentrenado para predecir las calificaciones a partir de las reseñas.  


In [ ]:
dataset = load_dataset("yelp_review_full")

dataset

Veamos un registro de muestra del conjunto de datos:


In [ ]:
dataset["train"][100]

La etiqueta es la clave de la etiqueta de clase.


In [ ]:
dataset["train"][100]["label"]

también está el texto:

In [ ]:
dataset["train"][100]['text']

Puedes seleccionar un subconjunto de datos para reducir el tiempo de entrenamiento:


In [ ]:
dataset["train"] = dataset["train"].select([i for i in range(1000)])
dataset["test"] = dataset["test"].select([i for i in range(200)])

Hay dos campos de datos:  
- label: la etiqueta de la reseña  
- text: una cadena que contiene el cuerpo de la reseña del usuario  



#### Tokenización de datos

El siguiente paso es cargar un tokenizador de BERT para tokenizar, rellenar y truncar reseñas y así manejar secuencias de longitud variable:

In [ ]:
# Instancia un tokenizador usando el modelo BERT base cased
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# Define una función para tokenizar ejemplos
def tokenize_function(examples):
    # Tokeniza el texto usando el tokenizador
    # Aplica padding para asegurar que todas las secuencias tengan la misma longitud
    # Aplica truncamiento para limitar la longitud máxima de la secuencia
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Aplica la función de tokenización al conjunto de datos por lotes
tokenized_datasets = dataset.map(tokenize_function, batched=True)


Las claves en cada elemento de `tokenized_datasets` son 'label', 'text', 'input_ids', 'token_type_ids' y 'attention_mask'.

In [ ]:
tokenized_datasets['train'][0].keys()

Para aplicar la función de preprocesamiento en todo el conjunto de datos, vamos a usar el método `map`. Puedes acelerar la función `map` estableciendo `batched=True` para procesar varios elementos del conjunto de datos a la vez.

Dado que el modelo está construido sobre el framework PyTorch, es crucial preparar el conjunto de datos en un formato que PyTorch pueda procesar directamente. Sigue estos pasos para garantizar la compatibilidad:


In [ ]:
# Elimina la columna 'text' porque el modelo no acepta texto sin procesar como entrada
tokenized_datasets = tokenized_datasets.remove_columns(["text"])

# Renombra la columna 'label' a 'labels' porque el modelo espera un argumento llamado 'labels'
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")

# Establece el formato del conjunto de datos para que devuelva tensores de PyTorch en lugar de listas
tokenized_datasets.set_format("torch")

El resultado es un conjunto de tensores con las claves: 'labels', 'input_ids', 'token_type_ids', 'attention_mask'


In [ ]:
tokenized_datasets['train'][0].keys()

#### DataLoader


A continuación, crea un DataLoader para los conjuntos de datos de entrenamiento y prueba para que puedas iterar sobre lotes de datos:


In [ ]:
# Crea un DataLoader para entrenamiento
train_dataloader = DataLoader(tokenized_datasets["train"], shuffle=True, batch_size=2)

# Crea un DataLoader para evaluación
eval_dataloader = DataLoader(tokenized_datasets["test"], batch_size=2)

### **Entrenar el modelo**


En esta sección, aprenderemos a crear el bucle de entrenamiento desde cero sin la ayuda de la clase `Trainer` de Hugging Face.

Cargaremos un modelo de clasificación preentrenado con 5 clases:


In [ ]:
# Instancia un modelo de clasificación de secuencias
modelo = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

#### Optimizador y programación de la tasa de aprendizaje

Vamos a crear un optimizador y un planificador de tasa de aprendizaje para ajustar el modelo. Puedes usar el optimizador AdamW de PyTorch:


In [ ]:
# Define el optimizador
optimizer = AdamW(modelo.parameters(), lr=5e-4)

# Establece el número de épocas
num_epochs = 10

# Calcula el número total de pasos de entrenamiento
num_training_steps = num_epochs * len(train_dataloader)

# Define el planificador de tasa de aprendizaje
lr_scheduler = LambdaLR(
    optimizer,
    lr_lambda=lambda current_step: (1 - current_step / num_training_steps)
)

Comprueba si CUDA está disponible y, de ser así, establecer el dispositivo correspondiente.


In [ ]:
# Comprueba si CUDA está disponible y, de ser así, establecer el dispositivo correspondiente
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Mueve el modelo al dispositivo apropiado
modelo.to(device)


#### **Bucle de entrenamiento**
 
La función `train_model` entrena un modelo usando un conjunto de datos de entrenamiento proporcionado a través de un dataloader. Comienza configurando una barra de progreso para monitorear visualmente el entrenamiento. El modelo se pone en modo de entrenamiento, lo cual es necesario para que comportamientos como dropout funcionen correctamente. 

La función procesa los datos en lotes por cada época, lo que implica varios pasos para cada lote: transferir los datos al dispositivo correcto (por ejemplo, GPU), pasarlos por el modelo para obtener salidas y calcular la pérdida, actualizar los parámetros del modelo usando los gradientes calculados, ajustar la tasa de aprendizaje y limpiar los gradientes anteriores. 

Estos pasos se repiten para cada lote de datos, y la barra de progreso se actualiza para reflejar el avance. Una vez completadas todas las épocas, el modelo entrenado se guarda para usarse después.










In [ ]:
def train_model(modelo, tr_dataloader):
    # Crea una barra de progreso para seguir el progreso del entrenamiento
    progress_bar = tqdm(range(num_training_steps))

    # Pone el modelo en modo entrenamiento
    modelo.train()
    tr_losses = []

    # Bucle de entrenamiento
    for epoch in range(num_epochs):
        total_loss = 0
        # Itera sobre los lotes de datos de entrenamiento
        for batch in tr_dataloader:
            # Mueve el lote al dispositivo apropiado
            batch = {k: v.to(device) for k, v in batch.items()}
            # Paso forward por el modelo
            outputs = modelo(**batch)
            # Calcula la pérdida
            loss = outputs.loss
            # Paso backward (calcular gradientes)
            loss.backward()

            total_loss += loss.item()
            # Actualiza los parámetros del modelo
            optimizer.step()
            # Actualiza el planificador de tasa de aprendizaje
            lr_scheduler.step()
            # Limpia los gradientes
            optimizer.zero_grad()
            # Actualiza la barra de progreso
            progress_bar.update(1)

        tr_losses.append(total_loss / len(tr_dataloader))

    # Grafica la pérdida
    plt.plot(tr_losses)
    plt.title("Pérdida de entrenamiento")
    plt.xlabel("Época")
    plt.ylabel("Pérdida")
    plt.show()

#### **Evaluación**

La función `evaluate_model` funciona de forma similar a `train_model` pero se usa para evaluar el rendimiento del modelo en lugar de entrenarlo. 

Usa un dataloader para procesar datos en lotes, pone el modelo en modo evaluación para asegurar mediciones precisas, y desactiva el cálculo de gradientes ya que no se entrena. 

La función calcula predicciones para cada lote, actualiza una métrica de exactitud y, finalmente, imprime la precisión global tras procesar todos los lotes.

In [ ]:
def evaluate_model(modelo, evl_dataloader):
    # Crea una instancia de la métrica accuracy para clasificación multiclase con 5 clases
    metric = Accuracy(task="multiclass", num_classes=5).to(device)

    # Pone el modelo en modo evaluación
    modelo.eval()

    # Desactiva el cálculo de gradientes durante la evaluación
    with torch.no_grad():
        # Itera sobre los lotes de datos de evaluación
        for batch in evl_dataloader:
            # Mover el lote al dispositivo apropiado
            batch = {k: v.to(device) for k, v in batch.items()}

            # Paso forward por el modelo
            outputs = modelo(**batch)

            # Obtiene las etiquetas predichas
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)

            # Acumula predicciones y etiquetas para la métrica
            metric(predictions, batch["labels"])

    # Calcula la exactitud
    accuracy = metric.compute()

    # Imprimir la exactitud
    print("Exactitud:", accuracy.item())

Puedes ahora entrenar el modelo. Este proceso tomará bastante tiempo y se recomienda hacerlo solo si cuentas con los recursos necesarios. Descomenta el siguiente código para entrenar el modelo:

In [ ]:
# train_model(modelp=model0,tr_dataloader=train_dataloader)
# torch.save(modelo, 'modelo1.pt')

![loss_gpt.png](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/HausLW2F_w30s1UK0zj7mQ/training-loss-BERT-Classification.png)


Ya estás listo para aprender a ajustar un modelo más complejo que pueda generar conversaciones entre un humano y un asistente.

#### Cargar el modelo guardado

Si quieres omitir el entrenamiento y cargar el modelo que entrenaste por 10 épocas, descomenta la siguiente celda:


In [ ]:
!wget 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/wFhKpkBMSgjmZKRSyayvsQ/bert-classification-model.pt'
modelo.load_state_dict(torch.load('bert-classification-model.pt',map_location=torch.device('cpu')))

Ahora puedes evaluar el modelo. Ten en cuenta que este proceso también tomará un tiempo:


In [ ]:
evaluate_model(modelo, eval_dataloader)

Ya estás listo para aprender a ajustar un modelo más complejo que genere conversaciones entre un humano y un asistente usando `SFTtrainer`.


**Ejercicio: Entrenamiento de un modelo conversacional usando SFTTrainer**

Explora cómo el ajuste fino de un transformer decodificador usando un conjunto de datos específico afecta la calidad de las respuestas generadas en una tarea de preguntas y respuestas.

1. **Paso 1**
    - Cargar la partición de entrenamiento (`train split`) del dataset `"timdettmers/openassistant-guanaco"` desde Hugging Face.

2. **Paso 2**
    - Cargar el modelo causal preentrenado `"facebook/opt-350m"` junto con su tokenizador desde Hugging Face.

3. **Paso 3**
    - Definir plantillas de instrucción y respuesta basadas en el formato del dataset de entrenamiento.

4. **Paso 4**
    - Crear un collator que prepare los datos en la forma adecuada para el entrenamiento, usando `DataCollatorForCompletionOnlyLM`.

5. **Paso 5**
    - Instanciar un objeto `SFTTrainer` proporcionando el modelo, el dataset y el collator.

6. **Paso 6**
    - Generar una respuesta inicial del modelo preentrenado para una pregunta de ejemplo.

7. **Paso 6A (Opcional)**
    - Entrenar el modelo usando el `trainer.train()`.

8. **Paso 6B**
    - Si no dispones de recursos suficientes, cargar el modelo ajustado desde el enlace proporcionado.

   ```python
    # !wget 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/Assistant_model.pt'
    # modelo.load_state_dict(torch.load('Assistant_model.pt',map_location=torch.device('cpu')))
    ```
9. **Paso 7**
    - Evaluar cómo responde el modelo ajustado a la misma pregunta especializada.


In [ ]:
## Tu respuesta